# Hello Python 출력 실습

In [11]:
print("Hello Python")

Hello Python


# If you know Numpy

In [12]:
import numpy as np

a = np.zeros((2, 2)); b = np.ones((2, 2))

np.sum(b, axis=1)


array([ 2.,  2.])

In [13]:
a.shape

(2, 2)

In [14]:
np.reshape(a, (1, 4))

array([[ 0.,  0.,  0.,  0.]])

# Repeat in TensorFlow

In [15]:
import tensorflow as tf
sess = tf.Session()

a = tf.zeros((2, 2)); b = tf.ones((2, 2))

sess.run(tf.reduce_sum(b, axis=1))

array([ 2.,  2.], dtype=float32)

In [16]:
a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

In [17]:
sess.run(tf.reshape(a, (1, 4)))

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

# sess.run()

In [18]:
import numpy as np

sess = tf.Session()
a = np.zeros((2, 2)); ta = tf.zeros((2, 2))

print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [19]:
print(ta)

Tensor("zeros_6:0", shape=(2, 2), dtype=float32)


In [20]:
print(sess.run(ta))

[[ 0.  0.]
 [ 0.  0.]]


In [21]:
sess = tf.Session()
a = np.zeros((2, 2)); ta = tf.zeros((2, 2))

print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [22]:
print(ta)

Tensor("zeros_7:0", shape=(2, 2), dtype=float32)


In [23]:
print(sess.run(ta))

[[ 0.  0.]
 [ 0.  0.]]


# Computation Graph

In [24]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

sess = tf.Session()
print(sess.run(c))

30.0


# TensorFlow Session

In [25]:
sess = tf.Session()
print(sess.run(c))

with tf.Session() as sess:
    print(sess.run(c))
    print(c.eval())

30.0
30.0
30.0


# TensorFlow Variables

In [28]:
w = tf.Variable(tf.zeros((2, 2)), name="weight")
with tf.Session() as sess:
    print(sess.run(w))

FailedPreconditionError: Attempting to use uninitialized value weight_1
	 [[Node: _send_weight_1_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=4453847917817088891, tensor_name="weight_1:0", _device="/job:localhost/replica:0/task:0/cpu:0"](weight_1)]]

# Variables need initialization!

In [32]:
w = tf.Variable(tf.random_normal([5, 2], stddev=0.1), name="weight")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[  1.21286772e-01  -1.32340118e-01]
 [  1.51039824e-01   4.26713452e-02]
 [  3.37811261e-02   1.74486995e-04]
 [ -1.30520776e-01   1.00652017e-01]
 [ -9.16759670e-02  -2.17691064e-01]]


# Updating Variables

In [33]:
state = tf.Variable(0, name="counter")
new_Value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_Value)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


# Fetching Variables

In [36]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3)
mul = tf.multiply(x1, temp)

with tf.Session() as sess:
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

5 5


# TensorFlow Placeholder

In [37]:
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

add = tf.add(a, b)
mul = tf.multiply(a, b)

with tf.Session() as sess:
    print(sess.run(add, feed_dict={a:2, b:3}))
    print(sess.run(mul, feed_dict={a:2, b:3}))

5
6


In [42]:
# using tf.constant
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.], [2.]])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    result sess.run(product)
    print(result)
    
# using placeholder
import numpy as np

matrix1 = tf.placeholder(tf.float32, [1, 2])
matrix2 = tf.placeholder(tf.float32, [2, 1])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.], [2.]])
    result = sess.run(product, feed_dict={matrix1: mv1, matrix2: mv2})
    print(result)

SyntaxError: invalid syntax (<ipython-input-42-093efb42cd7f>, line 7)